# Segmenting and Clustering Neighborhoods in Toronto

##### Importing libraries

In [2]:
import pandas as pd
!pip install geocoder
import geocoder # import geocoder

from geopy.geocoders import Nominatim 

import folium
import requests
import json
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Download and Explore Dataset

##### Importing Toronto's list of postal codes :

In [5]:

toronto_postal_code=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',flavor='bs4')

toronto_Neighborhoods=toronto_postal_code[0]



toronto_Neighborhoods.rename(columns={"Postal Code":"PostalCode"},inplace=True)
toronto_Neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Ignoring all the "Not assigned" boroughs :

In [6]:

toronto_Neighborhoods=toronto_Neighborhoods[toronto_Neighborhoods.Borough != "Not assigned"]
toronto_Neighborhoods.reset_index(drop=True, inplace=True)
toronto_Neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


##### Displaying toronto_Neighborhoods data frame size :

In [7]:
toronto_Neighborhoods.shape


(103, 3)

##### Importing coordinates from .csv file :

In [8]:
coordinates_df=pd.read_csv('https://cocl.us/Geospatial_data')

#coordinates_df.sort_values(by=['Postal Code'],inplace = True)
coordinates_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


##### Retreving coordinates for each postal code 

In [9]:
# Creating columns for the coordinates
toronto_Neighborhoods['Latitude']=""
toronto_Neighborhoods['Longitude']=""

# Retrieving coordinates values from coordinates_df
for i in range(len(toronto_Neighborhoods)):
    ind=coordinates_df[coordinates_df["Postal Code"]==toronto_Neighborhoods.iloc[i,0]].index.values[0]
    toronto_Neighborhoods['Latitude'][i]=round(coordinates_df['Latitude'][ind],6)
    toronto_Neighborhoods['Longitude'][i]=coordinates_df['Longitude'][ind]
   
    
toronto_Neighborhoods.head(10)


C:\Users\Red\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Red\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Red\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [8]:
toronto_Neighborhoods.shape

(103, 5)